In [8]:
import selenium 
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import WebDriverException
import requests
from bs4 import BeautifulSoup
import os
import urllib
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
!pip install fast-to-sql

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : r'C:\Users\A803377\Clearon DailySales'}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(executable_path=r"C://Driver/chromedriver.exe",chrome_options=chrome_options)

In [13]:
from selenium.webdriver.common.action_chains import ActionChains
# Login Cred
driver.get("https://clearoncorp-my.sharepoint.com/:f:/g/personal/brian_graham_clearon_com/Env-f9LZKm1OpeovMCHu3dgBLhIsp0AI28d3DC3SBqoC6w?e=LnWyPk")
time.sleep(2)
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "txtTOAAEmail")))
driver.find_element(By.ID,"txtTOAAEmail").send_keys('Smedicharla@solenis.com')
driver.find_element(By.XPATH,'//input[@id="txtTOAAEmail"]').click()
time.sleep(2)
ActionChains(driver).send_keys(Keys.SHIFT, Keys.ENTER).perform()
# driver.refresh()

In [14]:
# OTP Enter
ActionChains(driver).send_keys(Keys.SHIFT, Keys.ENTER).perform()

In [16]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
# Daily Sales Report
# WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//button[@title="Daily Sales Report"]'))).click()
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//button[@title="Open Order Analysis"]'))).click()
time.sleep(2)
# Download
button =driver.find_elements(By.XPATH,'//div[@data-automationid="ListCell"]')[-1]
action=ActionChains(driver)
action.context_click(button).perform()
time.sleep(3)
if driver.find_element(By.XPATH,'//ul[@role="presentation"]/li[7]').text =='Download':
    button_DL = driver.find_element(By.XPATH,'//ul[@role="presentation"]/li[7]')
time.sleep(3)
action=ActionChains(driver)
action.click(button_DL).perform()
time.sleep(3)

In [9]:
# Reading Path of Latest Excel file 
import glob
import os.path

folder_path = r'C:\Users\A803377\Clearon DailySales'
file_type = r'\*xlsx'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

print(max_file)

C:\Users\A803377\Clearon DailySales\CY Actuals Sales Report_3_14_2023.xlsx


In [11]:
# # Daily sales 

from datetime import timedelta
df=pd.read_excel(max_file,skiprows=3)
# df=pd.read_excel(r'C:\Users\A803377\Clearon DailySales\CY Actuals Sales Report_11_10_2022.xlsx',skiprows=3)

# df=dfs[list(dfs)[7]].copy()

df.dropna(subset=["Business Group", "'Material #"],inplace= True)
# File Operations
df[['Month','Year',"'Customer #","'Material #","'Billing document No.","Sales Doc #"]] = df[['Month','Year',"'Customer #","'Material #","'Billing document No.","Sales Doc #"]].astype(float).astype('string').replace('\.0', '', regex=True)
df["'Material #"] = df["'Material #"].apply(lambda x: '{0:0>18}'.format(x))
df["'Fiscal year/period"] = df["'Fiscal year/period"].apply(lambda x: '{0:0>8}'.format(x))
df["Month"] = df["Month"].apply(lambda x: '{0:0>3}'.format(x))
df["Year"] = df["Year"].apply(lambda x: '{0:0>4}'.format(x))
# df["'Actual Qty"] = df["'Actual Qty"].astype(float)
# Date
from datetime import datetime, timedelta
# df['Reporting_Date'] = (pd.Timestamp.today().strftime('%Y-%m-%d')) 
# Other Dates
df['Reporting_Date'] = ((pd.Timestamp.today()- timedelta(days=0)).strftime('%Y-%m-%d')) 
df['Reporting_Date'] =  pd.to_datetime(df['Reporting_Date'], format='%Y-%m-%d')
df[["Business Group","Product Element", "'Customer #", "'Customer","YTD", "'Material #", "'Material", "'Product Family", "'Billing document No.","'Fiscal year/period", "'Month", "Customer PO", "Sales Doc #","Year", "Month","Category","Tolling SKU", "InterCo Sales","Hierarchy Corrected","SBU", "Level 1 - Division", "Level 2 - Product Line","Level 3 - Group", "Level 4 - Brand Name", "Level 5 - Product Family","Level 6 - Segment", "Level 7 - Sub-Segment"]]= df[["Business Group","Product Element", "'Customer #", "'Customer","YTD", "'Material #", "'Material", "'Product Family", "'Billing document No.","'Fiscal year/period", "'Month", "Customer PO", "Sales Doc #","Year", "Month","Category","Tolling SKU", "InterCo Sales","Hierarchy Corrected","SBU", "Level 1 - Division", "Level 2 - Product Line","Level 3 - Group", "Level 4 - Brand Name", "Level 5 - Product Family","Level 6 - Segment", "Level 7 - Sub-Segment"]].astype('string')

# Using DataFrame.drop() method.
df=df.drop(df.columns[[22,28,38]], axis=1)
df = df.replace(np.nan, '', regex=True)


import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import urllib
import sqlalchemy as sa
import pyodbc
import time
from fast_to_sql import fast_to_sql as fts

start = time.time()
#Connecting to SQL 
conn = pyodbc.connect("""
DRIVER={SQL Server Native Client 11.0};
SERVER= NAAZUREEDWS01;
UID= svc_tableau_dev;
PWD= EwSol@321$;
DATABASE=TableauDB;
trusted_connection=Yes""")

# fts.fast_to_sql(df,'rpt.ManualClearonDailySales',conn, if_exists="append", custom={'Business Group':'nvarchar(225)','Product Element':'nvarchar(225)',
#                                                                                    "'Customer #":'nvarchar(225)',"'Customer":'nvarchar(225)',"'Material #":'nvarchar(225)',"'Material":'nvarchar(225)',
#                                                                                    "'Product Family":'nvarchar(225)',"'Billing document No.":'nvarchar(225)',"'Fiscal year/period":'nvarchar(225)',
#                                                                                   "'Month":'nvarchar(225)',"Customer PO":'nvarchar(225)',"Sales Doc #":'nvarchar(225)',
#                                                                                   "Year":'nvarchar(225)',"Month":'nvarchar(225)',"Category":'nvarchar(225)',
#                                                                                   "YTD":'nvarchar(225)',"Consolidated":'nvarchar(225)',"Prod. Type":'nvarchar(225)'}, temp=False)

fts.fast_to_sql(df,'rpt.ManualClearonDailySales',conn, if_exists="append", custom={'Business Group':'nvarchar(225)','Product Element':'nvarchar(225)',
                                                                                   "'Customer #":'nvarchar(225)',"'Customer":'nvarchar(225)',"'Material #":'nvarchar(225)',"'Material":'nvarchar(225)',
                                                                                   "'Product Family":'nvarchar(225)',"'Billing document No.":'nvarchar(225)',"'Fiscal year/period":'nvarchar(225)',
                                                                                  "'Month":'nvarchar(225)',"Customer PO":'nvarchar(225)',"Sales Doc #":'nvarchar(225)',
                                                                                  "Year":'nvarchar(225)',"Month":'nvarchar(225)',"Category":'nvarchar(225)',
                                                                                  "YTD":'nvarchar(225)',"Consolidated":'nvarchar(225)',"Prod. Type":'nvarchar(225)',"Tolling SKU":'nvarchar(225)', "InterCo Sales":'nvarchar(225)',"Hierarchy Corrected":'nvarchar(225)',"SBU":'nvarchar(225)',
                                                                                    "Level 1 - Division":'nvarchar(225)', "Level 2 - Product Line":'nvarchar(225)',"Level 3 - Group":'nvarchar(225)', 
                                                                                "Level 4 - Brand Name":'nvarchar(225)', "Level 5 - Product Family":'nvarchar(225)',"Level 6 - Segment":'nvarchar(225)',
                                                                                    "Level 7 - Sub-Segment":'nvarchar(225)'}, temp=False)


conn.commit()
conn.close()
end = time.time()
delta = end - start
print("took %.2f seconds to process" % delta)
print("Update Completed")
print(df['[Reporting_Date]'].unique())
print('Last File Uploaded :',max_file)
df

took 6.44 seconds to process
Update Completed
['2023-03-14T00:00:00.000000000']
Last File Uploaded : C:\Users\A803377\Clearon DailySales\CY Actuals Sales Report_3_14_2023.xlsx


,[Business_Group],[Product_Element],['Customer_#],['Customer],['Material_#],['Material],['Product_Family],['Billing_document_No.],['Fiscal_year/period],['Month],[Customer_PO],[Sales_Doc_#],['Actual_Qty],['Actual_Sls],['Total_COGM],[Year],[Month],[Category],[YTD],[Std._Margin],[Consolidated],[Prod._Type],[Reporting_Volume],[Reporting_Revenue],[Material_Cost],[Tolling_SKU],[InterCo_Sales],[Hierarchy_Corrected],[SBU],[Level_1_-_Division],[Level_2_-_Product_Line],[Level_3_-_Group],[Level_4_-_Brand_Name],[Level_5_-_Product_Family],[Level_6_-_Segment],[Level_7_-_Sub-Segment],[Less_Tolling_Volume],[Less_Interco_Volume],[Reporting_Date]
0,Clearon Products,Ancillaries,115244,SAM'S CLUB,000000000012002720,MEMBERS MARK 2X1GAL 40% ALGAECIDE,Members Mark,50399296,003.2023,MAR,5878934811,1372007,6426.0,16696.26,7711.20,2023,003,Rec Water,,8985.06,WALMART / SAM'S CLUB,Core Ancilliaries,6426.0,16696.26,7711.20,,,NEWC6000000A19474316,928.0,Pool Solutions-Clearon,Pool Maintenance,Other,,Specialty,Coagulants,Coagulants Liquid,6426.0,6426.0,2023-03-14
1,Clearon Products,Ancillaries,115244,SAM'S CLUB,000000000012002720,MEMBERS MARK 2X1GAL 40% ALGAECIDE,Members Mark,50399297,003.2023,MAR,1828873477,1372011,8568.0,22261.68,10281.60,2023,003,Rec Water,,11980.08,WALMART / SAM'S CLUB,Core Ancilliaries,8568.0,22261.68,10281.60,,,NEWC6000000A19474316,928.0,Pool Solutions-Clearon,Pool Maintenance,Other,,Specialty,Coagulants,Coagulants Liquid,8568.0,8568.0,2023-03-14
2,Clearon Products,CDB 56,115244,SAM'S CLUB,000000000012002480,MEMBERS MARK 24X1# SHOCK NEW,Members Mark,50399257,003.2023,MAR,1778186929,1371923,4320.0,16954.20,4338.40,2023,003,Rec Water,,12615.80,WALMART / SAM'S CLUB,Core ISO's,4320.0,16954.20,4338.40,,,NEWC5000000A18D89413,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,SDCC,SDCC Granular,SDCC Granular 56%,4320.0,4320.0,2023-03-14
3,Clearon Products,CDB 56,115244,SAM'S CLUB,000000000012002480,MEMBERS MARK 24X1# SHOCK NEW,Members Mark,50399259,003.2023,MAR,2327405551,1371957,1080.0,4238.55,1084.60,2023,003,Rec Water,,3153.95,WALMART / SAM'S CLUB,Core ISO's,1080.0,4238.55,1084.60,,,NEWC5000000A18D89413,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,SDCC,SDCC Granular,SDCC Granular 56%,1080.0,1080.0,2023-03-14
4,Clearon Products,CDB 56,115244,SAM'S CLUB,000000000012002480,MEMBERS MARK 24X1# SHOCK NEW,Members Mark,50399260,003.2023,MAR,4428087618,1371998,1080.0,4238.55,1084.60,2023,003,Rec Water,,3153.95,WALMART / SAM'S CLUB,Core ISO's,1080.0,4238.55,1084.60,,,NEWC5000000A18D89413,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,SDCC,SDCC Granular,SDCC Granular 56%,1080.0,1080.0,2023-03-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Clearon Products,CDB 90,120911,INNOVATIVE WATER CARE LLC,000000000012000030,CDB-90 COARSE 2205 LB SACK,CLEARON,50399312,003.2023,MAR,4534072345,1372019,41895.0,83371.05,37924.18,2023,003,Industrial Specialties,,45446.87,SOLENIS LLC,Core ISO's,0.0,0.00,0.00,,Yes,NEWC5000000A20D96435,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,TCCA,TCCA Granular,TCCA Pure Gr,41895.0,0.0,2023-03-14
113,Clearon Products,CDB 90,120911,INNOVATIVE WATER CARE LLC,000000000012000030,CDB-90 COARSE 2205 LB SACK,CLEARON,50399313,003.2023,MAR,4534072349,1372020,41895.0,83371.05,37924.18,2023,003,Industrial Specialties,,45446.87,SOLENIS LLC,Core ISO's,0.0,0.00,0.00,,Yes,NEWC5000000A20D96435,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,TCCA,TCCA Granular,TCCA Pure Gr,41895.0,0.0,2023-03-14
114,Clearon Products,CDB 90,124911,GORMAN HOME OFFICE,000000000012002210,NEW WAVE 50# CHLORINATING STICKS,New Wave,50399299,003.2023,MAR,287-P003112688,1372024,2400.0,8040.00,2621.30,2023,003,Rec Water,,5418.70,GORMAN,Core ISO's,2400.0,8040.00,2621.30,,,NEWC5000000A20D97437,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,TCCA,TCCA Sticks,TCCA Sticks Pure,2400.0,2400.0,2023-03-14
115,Clearon Products,CDB 90,124911,GORMAN HOME

# SS01 DS

In [25]:
# # Daily sales 

from datetime import timedelta
df=pd.read_excel(max_file,skiprows=3)
# df=pd.read_excel(r'C:\Users\A803377\Clearon DailySales\CY Actuals Sales Report_11_10_2022.xlsx',skiprows=3)

# df=dfs[list(dfs)[7]].copy()

df.dropna(subset=["Business Group", "'Material #"],inplace= True)
# File Operations
df[['Month','Year',"'Customer #","'Material #","'Billing document No.","Sales Doc #"]] = df[['Month','Year',"'Customer #","'Material #","'Billing document No.","Sales Doc #"]].astype(float).astype('string').replace('\.0', '', regex=True)
df["'Material #"] = df["'Material #"].apply(lambda x: '{0:0>18}'.format(x))
df["'Fiscal year/period"] = df["'Fiscal year/period"].apply(lambda x: '{0:0>8}'.format(x))
df["Month"] = df["Month"].apply(lambda x: '{0:0>3}'.format(x))
df["Year"] = df["Year"].apply(lambda x: '{0:0>4}'.format(x))
# df["'Actual Qty"] = df["'Actual Qty"].astype(float)
# Date
from datetime import datetime, timedelta
# df['Reporting_Date'] = (pd.Timestamp.today().strftime('%Y-%m-%d')) 
# Other Dates
df['Reporting_Date'] = ((pd.Timestamp.today()- timedelta(days=1)).strftime('%Y-%m-%d')) 
df['Reporting_Date'] =  pd.to_datetime(df['Reporting_Date'], format='%Y-%m-%d')
df[["Business Group","Product Element", "'Customer #", "'Customer","YTD", "'Material #", "'Material", "'Product Family", "'Billing document No.","'Fiscal year/period", "'Month", "Customer PO", "Sales Doc #","Year", "Month","Category","Tolling SKU", "InterCo Sales","Hierarchy Corrected","SBU", "Level 1 - Division", "Level 2 - Product Line","Level 3 - Group", "Level 4 - Brand Name", "Level 5 - Product Family","Level 6 - Segment", "Level 7 - Sub-Segment"]]= df[["Business Group","Product Element", "'Customer #", "'Customer","YTD", "'Material #", "'Material", "'Product Family", "'Billing document No.","'Fiscal year/period", "'Month", "Customer PO", "Sales Doc #","Year", "Month","Category","Tolling SKU", "InterCo Sales","Hierarchy Corrected","SBU", "Level 1 - Division", "Level 2 - Product Line","Level 3 - Group", "Level 4 - Brand Name", "Level 5 - Product Family","Level 6 - Segment", "Level 7 - Sub-Segment"]].astype('string')

# Using DataFrame.drop() method.
df=df.drop(df.columns[[22,28,38]], axis=1)
df = df.replace(np.nan, '', regex=True)


import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import urllib
import sqlalchemy as sa
import pyodbc
import time
from fast_to_sql import fast_to_sql as fts

start = time.time()
#Connecting to SQL 
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=10.26.10.7;"
            "Database=TableauDB;"
            "Trusted_Connection=Yes;")
cnxn = pyodbc.connect(cnxn_str)

# fts.fast_to_sql(df,'rpt.ManualClearonDailySales',conn, if_exists="append", custom={'Business Group':'nvarchar(225)','Product Element':'nvarchar(225)',
#                                                                                    "'Customer #":'nvarchar(225)',"'Customer":'nvarchar(225)',"'Material #":'nvarchar(225)',"'Material":'nvarchar(225)',
#                                                                                    "'Product Family":'nvarchar(225)',"'Billing document No.":'nvarchar(225)',"'Fiscal year/period":'nvarchar(225)',
#                                                                                   "'Month":'nvarchar(225)',"Customer PO":'nvarchar(225)',"Sales Doc #":'nvarchar(225)',
#                                                                                   "Year":'nvarchar(225)',"Month":'nvarchar(225)',"Category":'nvarchar(225)',
#                                                                                   "YTD":'nvarchar(225)',"Consolidated":'nvarchar(225)',"Prod. Type":'nvarchar(225)'}, temp=False)

fts.fast_to_sql(df,'rpt.ManualClearonDailySales',cnxn, if_exists="append", custom={'Business Group':'nvarchar(225)','Product Element':'nvarchar(225)',
                                                                                   "'Customer #":'nvarchar(225)',"'Customer":'nvarchar(225)',"'Material #":'nvarchar(225)',"'Material":'nvarchar(225)',
                                                                                   "'Product Family":'nvarchar(225)',"'Billing document No.":'nvarchar(225)',"'Fiscal year/period":'nvarchar(225)',
                                                                                  "'Month":'nvarchar(225)',"Customer PO":'nvarchar(225)',"Sales Doc #":'nvarchar(225)',
                                                                                  "Year":'nvarchar(225)',"Month":'nvarchar(225)',"Category":'nvarchar(225)',
                                                                                  "YTD":'nvarchar(225)',"Consolidated":'nvarchar(225)',"Prod. Type":'nvarchar(225)',"Tolling SKU":'nvarchar(225)', "InterCo Sales":'nvarchar(225)',"Hierarchy Corrected":'nvarchar(225)',"SBU":'nvarchar(225)',
                                                                                    "Level 1 - Division":'nvarchar(225)', "Level 2 - Product Line":'nvarchar(225)',"Level 3 - Group":'nvarchar(225)', 
                                                                                "Level 4 - Brand Name":'nvarchar(225)', "Level 5 - Product Family":'nvarchar(225)',"Level 6 - Segment":'nvarchar(225)',
                                                                                    "Level 7 - Sub-Segment":'nvarchar(225)'}, temp=False)


cnxn.commit()
cnxn.close()
end = time.time()
delta = end - start
print("took %.2f seconds to process" % delta)
print("Update Completed")
print(df['[Reporting_Date]'].unique())
print('Last File Uploaded :',max_file)
df

took 17.70 seconds to process
Update Completed
['2023-02-21T00:00:00.000000000']
Last File Uploaded : C:\Users\A803377\Clearon DailySales\CY Actuals Sales Report_2_21_2023.xlsx


,[Business_Group],[Product_Element],['Customer_#],['Customer],['Material_#],['Material],['Product_Family],['Billing_document_No.],['Fiscal_year/period],['Month],[Customer_PO],[Sales_Doc_#],['Actual_Qty],['Actual_Sls],['Total_COGM],[Year],[Month],[Category],[YTD],[Std._Margin],[Consolidated],[Prod._Type],[Reporting_Volume],[Reporting_Revenue],[Material_Cost],[Tolling_SKU],[InterCo_Sales],[Hierarchy_Corrected],[SBU],[Level_1_-_Division],[Level_2_-_Product_Line],[Level_3_-_Group],[Level_4_-_Brand_Name],[Level_5_-_Product_Family],[Level_6_-_Segment],[Level_7_-_Sub-Segment],[Less_Tolling_Volume],[Less_Interco_Volume],[Reporting_Date]
0,Clearon Products,Ancillaries,115244,SAM'S CLUB,000000000012002720,MEMBERS MARK 2X1GAL 40% ALGAECIDE,Members Mark,50399017,002.2023,FEB,8678773377,1371747,29988.0,77915.88,35985.60,2023,002,Rec Water,,41930.28,WALMART / SAM'S CLUB,Core Ancilliaries,29988.0,77915.88,35985.6,,,NEWC6000000A19474316,928.0,Pool Solutions-Clearon,Pool Maintenance,Other,,Specialty,Coagulants,Coagulants Liquid,29988.0,29988.0,2023-02-21
1,Clearon Products,Ancillaries,115244,SAM'S CLUB,000000000012002720,MEMBERS MARK 2X1GAL 40% ALGAECIDE,Members Mark,50399018,002.2023,FEB,7728743453,1371739,26775.0,69567.75,32130.00,2023,002,Rec Water,,37437.75,WALMART / SAM'S CLUB,Core Ancilliaries,26775.0,69567.75,32130.0,,,NEWC6000000A19474316,928.0,Pool Solutions-Clearon,Pool Maintenance,Other,,Specialty,Coagulants,Coagulants Liquid,26775.0,26775.0,2023-02-21
2,Clearon Products,Ancillaries,115244,SAM'S CLUB,000000000012002720,MEMBERS MARK 2X1GAL 40% ALGAECIDE,Members Mark,50399019,002.2023,FEB,8427478766,1371743,29988.0,77915.88,35985.60,2023,002,Rec Water,,41930.28,WALMART / SAM'S CLUB,Core Ancilliaries,29988.0,77915.88,35985.6,,,NEWC6000000A19474316,928.0,Pool Solutions-Clearon,Pool Maintenance,Other,,Specialty,Coagulants,Coagulants Liquid,29988.0,29988.0,2023-02-21
3,Clearon Products,Ancillaries,115244,SAM'S CLUB,000000000012002720,MEMBERS MARK 2X1GAL 40% ALGAECIDE,Members Mark,50399020,002.2023,FEB,7133563459,1371736,29988.0,77915.88,35985.60,2023,002,Rec Water,,41930.28,WALMART / SAM'S CLUB,Core Ancilliaries,29988.0,77915.88,35985.6,,,NEWC6000000A19474316,928.0,Pool Solutions-Clearon,Pool Maintenance,Other,,Specialty,Coagulants,Coagulants Liquid,29988.0,29988.0,2023-02-21
4,Clearon Products,Ancillaries,115244,SAM'S CLUB,000000000012002720,MEMBERS MARK 2X1GAL 40% ALGAECIDE,Members Mark,50399021,002.2023,FEB,5628804625,1371770,28917.0,75133.17,34700.40,2023,002,Rec Water,,40432.77,WALMART / SAM'S CLUB,Core Ancilliaries,28917.0,75133.17,34700.4,,,NEWC6000000A19474316,928.0,Pool Solutions-Clearon,Pool Maintenance,Other,,Specialty,Coagulants,Coagulants Liquid,28917.0,28917.0,2023-02-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Clearon Products,CDB 90,120911,INNOVATIVE WATER CARE LLC,000000000012000030,CDB-90 COARSE 2205 LB SACK,CLEARON,50399195,002.2023,FEB,4534065162,1371911,41895.0,83371.05,37924.18,2023,002,Industrial Specialties,,45446.87,SOLENIS LLC,Core ISO's,0.0,0.00,0.0,,Yes,NEWC5000000A20D96435,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,TCCA,TCCA Granular,TCCA Pure Gr,41895.0,0.0,2023-02-21
296,Clearon Products,CDB 90,120911,INNOVATIVE WATER CARE LLC,000000000012000030,CDB-90 COARSE 2205 LB SACK,CLEARON,50399196,002.2023,FEB,4534065164,1371912,41895.0,83371.05,37924.19,2023,002,Industrial Specialties,,45446.86,SOLENIS LLC,Core ISO's,0.0,0.00,0.0,,Yes,NEWC5000000A20D96435,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,TCCA,TCCA Granular,TCCA Pure Gr,41895.0,0.0,2023-02-21
297,Clearon Products,CDB 90,120911,INNOVATIVE WATER CARE LLC,000000000012000030,CDB-90 COARSE 2205 LB SACK,CLEARON,50399202,002.2023,FEB,4534064598,1371905,41895.0,83371.05,37924.18,2023,002,Industrial Specialties,,45446.87,SOLENIS LLC,Core ISO's,0.0,0.00,0.0,,Yes,NEWC5000000A20D96435,927.0,Pool Solutions-Clearon,Isocyanurates,Other,,

In [83]:
# df.info()

In [10]:
from datetime import timedelta
print((pd.Timestamp.today()- timedelta(days=0)).strftime('%Y-%m-%d')) 

2023-03-14


--------------------------------- Daily Sales-----------------------------------------------------

In [8]:
# import pandas as pd
# import glob

# # getting excel files from Directory Desktop
# path = r'C:\Users\A803377\Clearon Project'

# # read all the files with extension .xlsx i.e. excel 
# filenames = glob.glob(path + "\*.xlsx")
# print('File names:', filenames)

# dfs = {}

# # for loop to iterate all excel files 
# # loop over the list of csv files
# for f in filenames:
    
#     # read the csv file
#     dfs[f.split("\\")[-1]] = pd.read_excel(f,skiprows=3)
      
#     # print the location and filename
#     print('Location:', f)
#     print('File Name:', f.split("\\")[-1],'\n')

In [9]:
# dfs.keys()

In [10]:
# df=dfs[list(dfs)[1]].copy()
# df

In [11]:
# list(dfs)[7]

In [2]:
dfs = []

In [12]:
# import pandas as pd
# import glob

# # getting excel files from Directory Desktop
# path = r'C:\Users\A803377\Clearon OpenOrders'

# # read all the files with extension .xlsx i.e. excel 
# filenames = glob.glob(path + "\*.xlsx")
# print('File names:', filenames)

# dfs = {}

# # for loop to iterate all excel files 
# # loop over the list of csv files
# for f in filenames:
    
#     # read the csv file
#     dfs[f.split("\\")[-1]] = pd.read_excel(f)
      
#     # print the location and filename
#     print('Location:', f)
#     print('File Name:', f.split("\\")[-1],'\n')

In [5]:
# Reading Path of Latest Excel file 
import glob
import os.path

folder_path = r'C:\Users\A803377\Clearon OpenOrders'
file_type = r'\*xlsx'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

print(max_file)

C:\Users\A803377\Clearon OpenOrders\Open Order Analysis 3_10_2023.xlsx


In [6]:
from datetime import timedelta

df=pd.read_excel(max_file)
# df=pd.read_excel(r"C:\Users\A803377\Clearon OpenOrders\Open Order Analysis 11_10_2022.xlsx")

# df=dfs[list(dfs)[3]].copy()
df[["Plant","Delivery","Ship-to party","Sold-to party","Sales Document"]] = df[["Plant","Delivery","Ship-to party","Sold-to party","Sales Document"]].astype(float).astype('string').replace('\.0', '', regex=True)
df[['Created by','Name of sold-to party', 'Name of ship-to party','Purchase order no.','Material', 'Description', 'Sales unit','Status','Delivery block description','Shipping Point/Receiving Pt']] = df[['Created by','Name of sold-to party', 'Name of ship-to party','Purchase order no.','Material', 'Description', 'Sales unit','Status','Delivery block description','Shipping Point/Receiving Pt']].astype('string')
df["Plant"] = df["Plant"].apply(lambda x: '{0:0>4}'.format(x))
df[["Plant"]] = df[["Plant"]].astype('string')

df["Order Quantity"]= df["Order Quantity"].astype(float)
# df['Reporting_Date'] = (pd.Timestamp.today().strftime('%Y-%m-%d')) 

df['Reporting_Date'] = ((pd.Timestamp.today()- timedelta(days=3)).strftime('%Y-%m-%d')) 
df['Reporting_Date'] =  pd.to_datetime(df['Reporting_Date'], format='%Y-%m-%d')

df = df.replace(np.nan, '', regex=True)
# df[['Goods Issue Date','Act. Gds Mvmnt Date','Requested deliv.date','Created on']] = df[['Goods Issue Date','Act. Gds Mvmnt Date','Requested deliv.date','Created on']].astype('datetime64[ns]')

df

,Created on,Created by,Plant,Sales Document,Sold-to party,Name of sold-to party,Name of ship-to party,Purchase order no.,Material,Description,Sales unit,Order Quantity,Net weight,Net price,Status,Delivery,Delivery block description,Requested deliv.date,Act. Gds Mvmnt Date,Net value,Ship-to party,Shipping Point/Receiving Pt,Goods Issue Date,Reporting_Date
0,2020-12-02,BKGRAH,2700,1363542,115151,BIOLAB INCORPORATED,BIO-LAB INC,4500891733,12000032,CDB-90 COARSE 2205 LB SACK,SS,19.0,41895.0,3941.16,POSTED TO ACCTG,450516295,,2022-03-17,2021-09-15,74882.0,121663,CLE1,2021-09-10,2023-03-10
1,2020-12-02,BKGRAH,2700,1363542,115151,BIOLAB INCORPORATED,BIO-LAB INC,4500891733,12000032,CDB-90 COARSE 2205 LB SACK,SS,19.0,41895.0,3959.05,POSTED TO ACCTG,450516296,,2022-03-17,2021-09-10,75222.0,121663,CLE1,2021-09-10,2023-03-10
2,2020-12-02,BKGRAH,2700,1363542,115151,BIOLAB INCORPORATED,BIO-LAB INC,4500891733,12000032,CDB-90 COARSE 2205 LB SACK,SS,19.0,41895.0,3914.58,POSTED TO ACCTG,450518019,,2022-03-17,2022-03-17,74377.0,121663,CLE1,2022-03-18,2023-03-10
3,2020-12-02,BKGRAH,2700,1363542,115151,BIOLAB INCORPORATED,BIO-LAB INC,4500891733,12000032,CDB-90 COARSE 2205 LB SACK,SS,19.0,41895.0,3914.58,POSTED TO ACCTG,450518020,,2022-03-17,2022-03-17,74377.0,121663,CLE1,2022-03-18,2023-03-10
4,2020-12-02,BKGRAH,2700,1363545,115151,BIOLAB INCORPORATED,BIO-LAB INC,4500891736,12000032,CDB-90 COARSE 2205 LB SACK,SS,19.0,41895.0,3914.84,POSTED TO ACCTG,450518132,,2022-03-24,2022-03-25,74382.0,121663,CLE1,2022-03-24,2023-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,2023-03-09,KSGRAH,2700,1372106,115194,LESLIE'S POOLMART INC,LESLIE'S POOLMART,11411426,12002864,"LESLIE'S 35# 3"" 8OZ TAB-UW LABELED",EA,183.0,6405.0,0.19,OPEN DELIVERY,450521592,,2023-03-15,NaT,0.0,121571,CLE2,2023-03-14,2023-03-10
3696,2023-03-09,KSGRAH,2700,1372107,115194,LESLIE'S POOLMART INC,LESLIE'S POOLMART,11411427,12002880,IN THE SWIM 50# 8OZ TAB-PE LTD QTY,CS,50.0,2500.0,330.00,OPEN DELIVERY,450521593,,2023-03-15,NaT,0.0,121571,CLE2,2023-03-14,2023-03-10
3697,2023-03-09,KSGRAH,2700,1372108,102960,ECOLAB INC,ECOLAB INC,5502974782,12000794,TRICHLOR-O-CIDE 5600 4X10#,CS,24.0,960.0,186.17,DELIVERY BLOCK,,Pending Availability,2023-06-07,NaT,0.0,116200,CLE2,NaT,2023-03-10
3698,2023-03-09,KSGRAH,2700,1372109,102960,ECOLAB INC,ECOLAB INC,5502974783,12000794,TRICHLOR-O-CIDE 5600 4X10#,CS,24.0,960.0,186.17,DELIVERY BLOCK,,Pending Availability,2023-07-03,NaT,0.0,116200,CLE2,NaT,2023-03-10


In [2]:
# df.columns

In [25]:
# df.info()

In [7]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import urllib
import sqlalchemy as sa
import pyodbc
import time
import pytds
from fast_to_sql import fast_to_sql as fts

start = time.time()
#Connecting to SQL 
conn = pyodbc.connect("""
DRIVER={SQL Server Native Client 11.0};
SERVER= NAAZUREEDWS01;
UID= svc_tableau_dev;
PWD= EwSol@321$;
DATABASE=TableauDB;
trusted_connection=Yes""")

fts.fast_to_sql(df,'rpt.ManualClearonOpenOrders',conn, if_exists="append", custom={'Created by':'nvarchar(225)','Plant':'nvarchar(225)',
                                                                                   "Sales Document":'nvarchar(225)',"Sold-to party":'nvarchar(225)',"Name of sold-to party":'nvarchar(225)',"Name of ship-to party":'nvarchar(225)',
                                                                                   "Purchase order no.":'nvarchar(225)',"Material":'nvarchar(225)',"Description":'nvarchar(225)',
                                                                                  "Sales unit":'nvarchar(225)',"Status":'nvarchar(225)',"Delivery":'nvarchar(225)',
                                                                                  "Delivery block description":'nvarchar(225)',"Shipping Point/Receiving Pt":'nvarchar(225)',"Ship-to party":'nvarchar(225)'
                                                                                  ,"Goods Issue Date":'datetime2(7)',"Act. Gds Mvmnt Date":'datetime2(7)',"Requested deliv.date":'datetime2(7)',"Created on":'datetime2(7)'}, temp=False)


conn.commit()
conn.close()



# cursor = conn.cursor()

# cursor.execute("""
# UPDATE TableauDB.[rpt].[ManualClearonOpenOrders] 
# SET [Act._Gds_Mvmnt_Date] = ''
# WHERE [Act._Gds_Mvmnt_Date] = '0001-01-01 00:00:00.0000000'
#                 """)

# conn.commit()
# conn.close()

end = time.time()
delta = end - start
print("took %.2f seconds to process" % delta)
print("Update Completed")
print(df['[Reporting_Date]'].unique())
print('Last File Uploaded :',max_file)

took 11.03 seconds to process
Update Completed
['2023-03-10T00:00:00.000000000']
Last File Uploaded : C:\Users\A803377\Clearon OpenOrders\Open Order Analysis 3_10_2023.xlsx


In [21]:
# conn.close()
print(df['[Reporting_Date]'].unique())

['2022-11-21T00:00:00.000000000']


In [20]:
# df= pd.read_excel(r'C:\Users\A803377\Downloads\Zcrosstab Mapping.xlsx',sheet_name=1)
# df

In [21]:
# df[["Material Group","Spend Type"]] = df[["Material Group","Spend Type"]].astype('string')

# df.info()

In [22]:
# import pandas as pd
# import warnings
# warnings.filterwarnings("ignore")
# import urllib
# import sqlalchemy as sa
# import pyodbc
# import time
# from fast_to_sql import fast_to_sql as fts

# start = time.time()
# #Connecting to SQL 
# conn = pyodbc.connect("""
# DRIVER={SQL Server Native Client 11.0};
# SERVER= NAAZUREEDWS01;
# UID= svc_tableau_dev;
# PWD= EwSol@321$;
# DATABASE=TableauDB;
# trusted_connection=Yes""")

# # fts.fast_to_sql(df,'rpt.ManualClearonSpend_Mapping',conn, if_exists="append", custom={'Material Group':'nvarchar(225)','Spend Type':'nvarchar(225)'}, temp=False)

# conn.commit()
# conn.close()
# end = time.time()
# delta = end - start
# print("took %.2f seconds to process" % delta)
# print("Update Completed")

# SS01 OO

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import urllib
import sqlalchemy as sa
import pyodbc
import time
from fast_to_sql import fast_to_sql as fts

start = time.time()
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=10.26.10.7;"
            "UID= svc_tableau_dev;"
            "PWD= Onestream!@#;"
            "Database=TableauDB;"
            "Trusted_Connection=Yes;")
cnxn = pyodbc.connect(cnxn_str)


fts.fast_to_sql(df,'rpt.ManualClearonOpenOrders',cnxn, if_exists="append", custom={'Created by':'nvarchar(225)','Plant':'nvarchar(225)',
                                                                                   "Sales Document":'nvarchar(225)',"Sold-to party":'nvarchar(225)',"Name of sold-to party":'nvarchar(225)',"Name of ship-to party":'nvarchar(225)',
                                                                                   "Purchase order no.":'nvarchar(225)',"Material":'nvarchar(225)',"Description":'nvarchar(225)',
                                                                                  "Sales unit":'nvarchar(225)',"Status":'nvarchar(225)',"Delivery":'nvarchar(225)',
                                                                                  "Delivery block description":'nvarchar(225)',"Shipping Point/Receiving Pt":'nvarchar(225)',"Ship-to party":'nvarchar(225)'
                                                                                  ,"Goods Issue Date":'datetime2(7)',"Act. Gds Mvmnt Date":'datetime2(7)',"Requested deliv.date":'datetime2(7)',"Created on":'datetime2(7)'}, temp=False)





# cursor = cnxn.cursor()

# cursor.execute("""
# UPDATE TableauDB.[rpt].[ManualClearonOpenOrders] 
# SET [Act._Gds_Mvmnt_Date] = ''
# WHERE [Act._Gds_Mvmnt_Date] = '0001-01-01 00:00:00.0000000'
#                 """)

# conn.commit()
# conn.close()
cnxn.commit()
cnxn.close()
end = time.time()
delta = end - start
print("took %.2f seconds to process" % delta)
print("Update Completed")
# print(df['[Reporting_Date]'].unique())
# print('Last File Uploaded :',max_file)

OperationalError: ('08001', '[08001] [Microsoft][SQL Server Native Client 11.0]Unable to complete login process due to delay in opening server connection (0) (SQLDriverConnect)')